In [ ]:
from google.colab import files

# This will prompt you to select the kaggle.json file
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# !pip install kaggle
!kaggle competitions download -c bird-classification-competition



In [ ]:
!unzip -q bird-classification-competition.zip


In [ ]:
import pandas as pd
import os

# For CSV files
df = pd.read_csv('Upload/Upload/train.csv')
print(df.head())
print(os.getcwd())
!ls content/Upload/Upload

!ls 'content/Upload/Upload/valid/ABBOTTS BABBLER'


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim


In [ ]:
# normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Assuming datasets are stored in folders named 'train', 'test', 'val'
train_dataset = ImageFolder(root='/content/Upload/Upload/train', transform=transform)
val_dataset = ImageFolder(root='/content/Upload/Upload/valid', transform=transform)
test_dataset = ImageFolder(root='/content/Upload/Upload/test', transform=transform)

num_classes = len(test_dataset.class_to_idx)
print(f"Number of classes in the test dataset: {num_classes}")


# Create data loaders
batch_size = 64  # You can adjust this according to your GPU memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:

# Calculation:
# Input size: 224x224
# After 1st pooling: 112x112 (since stride=2)
# After 2nd pooling: 56x56
# After 3rd pooling: 28x28
# If the final convolutional layer has 64 output channels, then the total number of features going into the fully connected layer is 64 * 28 * 28.

import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=525):  # Setting the number of classes based on your dataset
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # Adjusted dimensions based on pooling layers
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)  # Output layer adjusted for 525 classes
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # Flatten the tensor for the fully connected layer
        x = x.view(-1, 64 * 28 * 28)  # Adjusted for the output size of the last pooling layer
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
# Example using a pre-trained ResNet model
model = torchvision.models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 525)  # Adjust the number of output classes to match your dataset


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = model.to(device)


In [ ]:
# Train the model
for epoch in range(10):  # Number of epochs can be adjusted
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)#If you're training on a GPU, you would set device = torch.device("cuda:0") to specify the first GPU as your device. Then, using .to(device) ensures all your computations happen on that GPU.
# For example : Let's say you're training a neural network to classify images of cats and dogs. Your training dataset consists of 1000 images, and you've set your batch size to 100. This means train_loader will yield 10 batches, each containing 100 images and their corresponding labels.
# In the first iteration of the loop, i is 0 (the first batch), and data contains the first batch of 100 images and their 100 labels.
# data[0] is a tensor of shape [100, C, H, W] (assuming each image has C channels, height H, and width W). This represents the 100 images in the batch.
# data[1] is a tensor of shape [100], representing the labels (e.g., 0 for cat, 1 for dog) for each of the 100 images.
        optimizer.zero_grad()#optimizer.zero_grad() ensures that the gradient calculations are correct for each batch by clearing the old gradients, preventing them from accumulating across batches.
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()# gradient of the loss function with respect to that parameter. This is done using the chain rule of calculus, working backwards from the output layer to the input layer (hence "backward pass").
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # Print average loss every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


model_save_name = 'classifier_11feb.pt'
path = F"/content/drive/My Drive/{model_save_name}"  # Adjust the path based on where you want to save in Google Drive
torch.save(model.state_dict(), path)


In [ ]:
import pandas as pd
import torch
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Load the validation dataset CSV file
val_csv_path = '/content/Upload/Upload/val.csv'  # Adjust this path as needed
val_df = pd.read_csv(val_csv_path)

# Create a dictionary mapping file paths to IDs
file_path_to_id = {row['filepaths']: row['ID'] for index, row in val_df.iterrows()}

# Assuming model, val_loader, and device are already defined
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
all_predictions = []
all_targets = []

# Define the prefix that needs to be removed from paths in val_loader to match the CSV
prefix_to_remove = '/content/Upload/Upload/'

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Accumulate all the predictions and labels
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Calculate and print accuracy
accuracy = 100 * correct / total
print(f'Accuracy of the network on the validation images: {accuracy:.2f}%')

# Calculate precision, recall, and F1 score using sklearn
precision = precision_score(all_targets, all_predictions, average='weighted')
recall = recall_score(all_targets, all_predictions, average='weighted')
f1 = f1_score(all_targets, all_predictions, average='weighted')

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!ls "/content/drive/My Drive/"


In [ ]:
from google.colab import drive
import torch
import torchvision.models as models

# Mount Google Drive
drive.mount('/content/drive')

# Automatically use GPU if available, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Specify the exact path to the saved model
model_path = '/content/drive/My Drive/classifier_11feb.pt'

# Initialize the pre-trained ResNet50 model architecture
model = models.resnet50(pretrained=True)

# If you modified the final layer of the model, replicate that modification
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 525)  # Adjust this to match the number of classes in your dataset

# Load the saved model weights. If using GPU, 'map_location' is not needed. It automatically maps to the current device.
model.load_state_dict(torch.load(model_path, map_location=device))

# Move the model to the selected device
model = model.to(device)

# Set the model to evaluation mode
model.eval()


In [ ]:
import pandas as pd
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from PIL import Image

# Define batch_size
# batch_size = 32  # Or any other size that fits your needs

# normalization--same as when training
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the validation dataset
val_dataset = datasets.ImageFolder(root='/content/Upload/Upload/test', transform=transform)

# Create data loaders
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load the CSV file containing the file paths
val_csv_path = '/content/Upload/Upload/test.csv'  # Adjust if necessary
val_df = pd.read_csv(val_csv_path)

# Copy the original filepaths to a new column for comparison
val_df['original_filepaths'] = val_df['filepaths']

# Replace the filepaths that start with the specified pattern
val_df['filepaths'] = val_df['filepaths'].str.replace(r'^test/PARAKETT\s+AKULET/', 'test/PARAKETT  AUKLET/', regex=True)

# Identify the filepaths that have changed
changed_filepaths = val_df[val_df['filepaths'] != val_df['original_filepaths']]

# Print all filepaths that changed
for index, row in changed_filepaths.iterrows():
    print(f"Original: {row['original_filepaths']} -> Updated: {row['filepaths']}")

# Save the changes back to val.csv
val_df.to_csv(val_csv_path, index=False)


# Set the base path where the images are stored
base_path = '/content/Upload/Upload'  # Replace with the base path to your image directories

# Prepare the submission DataFrame
submission_df = pd.DataFrame(columns=['ID', 'ClassificationID'])

# # Loop through the file paths in the val.csv and make predictions
for index, row in val_df.iterrows():
    # Construct the full image path
    image_path = os.path.join(base_path, row['filepaths'])

    try:
        # Open the image using PIL
        image = Image.open(image_path).convert('RGB')
        # Transform the image
        image = transform(image)
        # Add a batch dimension
        image = image.unsqueeze(0).to(device)
        # Predict the class
        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
            print([row['ID'], predicted.item()])
        # Append the ID and predicted class ID to the submission dataframe
        submission_df = submission_df.append({'ID': row['ID'], 'ClassificationID': predicted.item()}, ignore_index=True)

    except Exception as e:
        print(f"An error occurred with image {image_path}: {e}")

# Save the new submission DataFrame to a CSV file
submission_path = '/content/drive/My Drive/submission_11feb_new.csv'
submission_df.to_csv(submission_path, index=False)
print(f"New submission file saved to {submission_path}")


In [ ]:
# Make sure the submission DataFrame has the same length as val_df
assert len(submission_df) == len(val_df), "Submission length does not match"
# Sort the submission DataFrame based on the ID column to ensure correct order
submission_df.sort_values('ID', inplace=True)
# Reset the index to avoid any potential issues with indexing
submission_df.reset_index(drop=True, inplace=True)
# Save the submission DataFrame to a CSV file
submission_df.to_csv(submission_path, index=False)
print(f"Submission file saved to {submission_path}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


model_save_name = 'classifier_11feb.pt'
path = F"/content/drive/My Drive/{model_save_name}"  
torch.save(model.state_dict(), path)
